In [ ]:
import shutil
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.notebook import tqdm
import imageio
import ast


def load_mask(row):
    shape = (row.height, row.width, 3)
    mask = np.zeros(shape, dtype=np.uint8)

    for i, rle in enumerate(row.segmentation):
        if rle:
            mask[..., i] = rle_decode(rle, shape[:2])

    return mask * 255


# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape):
    """
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    """
    s = np.asarray(mask_rle.split(), dtype=int)
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction


def save_array(file_path, array):
    file_path = Path(file_path)
    file_path.parent.mkdir(parents=True, exist_ok=True)
    np.save(file_path, array)



def load_image(path):
    image = cv2.imread(path,cv2.IMREAD_UNCHANGED)
    #image = image.astype("uint")  # original is uint16
    #image /= image.max()
    return image


def load_images(paths):
    images = [load_image(path) for path in ast.literal_eval(paths)]
    images = np.stack(images, axis=-1)    
    
    return images


def load_mask(path):
    mask = np.load(path)
    #mask = mask.astype("float32")
    #mask /= 255.0
    return mask
    
    
def show_image(image, mask=None):
    plt.imshow(image, cmap="bone")

    if mask is not None:
        plt.imshow(mask, alpha=0.5)

        handles = [
            Rectangle((0, 0), 1, 1, color=_c) for _c in [(0.667, 0.0, 0.0), (0.0, 0.667, 0.0), (0.0, 0.0, 0.667)]
        ]
        labels = ["Stomach", "Large Bowel", "Small Bowel"]

        plt.legend(handles, labels)

    plt.axis("off")
    

def show_grid(train_df, nrows, ncols):
    fig, _ = plt.subplots(figsize=(5 * ncols, 5 * nrows))

    train_df_sampled = train_df[~train_df["empty"]].sample(n=nrows * ncols)
    for i, row in enumerate(train_df_sampled.itertuples()):

        #image = load_images(row.image_paths)
        image = load_image(row.image_path)

        mask = load_mask(row.mask_path)

        plt.subplot(nrows, ncols, i + 1)
        plt.tight_layout()
        plt.title(row.id)

        show_image(image, mask)
        
def try_tranform(train_df,nrows,tranform,image_flag = 1,mask_flag = 1,albumentation = 0):
    fig, _ = plt.subplots(figsize=(5 * 2, 5 * nrows))

    train_df_sampled = train_df[~train_df["empty"]].sample(n=nrows)
    counter = 1
    for i, row in enumerate(train_df_sampled.itertuples()):

        #image = load_images(row.image_paths)
        image = load_image(row.image_path)

        mask = load_mask(row.mask_path)

        plt.subplot(nrows, ncols, counter)
        plt.tight_layout()
        plt.title(row.id+' Original 2.5D')

        show_image(image, mask)
        counter+=1



        plt.subplot(nrows, ncols, counter)
        plt.tight_layout()
        plt.title(row.id+' After Transformation')
        
        if albumentation == 1:
            transformed = transform(image=image, mask=mask)
            show_image(transformed['image'], transformed['mask'])
        else:
            
        
            if image_flag == 1:
                transformed_image = transform(image)
            else:
                transformed_image = image

            if mask_flag == 1:
                transformed_mask = transform(mask)
            else:
                transformed_mask = mask
            
            show_image(transformed_image, transformed_mask)
        
        counter+=1
    

In [ ]:
KAGGLE_DIR = Path("/") / "kaggle"
INPUT_DIR = KAGGLE_DIR / "input"
OUTPUT_DIR = KAGGLE_DIR / "working"

INPUT_DATA_DIR = INPUT_DIR / "uw-madison-gi-tract-image-segmentation"
N_SPLITS = 5
RANDOM_SEED = 2022

# For 2.5D Data
CHANNELS = 3
STRIDE = 2

OUTPUT_DATA_DIR = OUTPUT_DIR / INPUT_DATA_DIR.stem
OUTPUT_DATA_DIR.mkdir(exist_ok=True)

DEBUG = True

In [ ]:
df = pd.read_csv(INPUT_DIR / 'uw-madison-gi-tract-image-segmentation-masks'/'train_preprocessed.csv')

In [ ]:
nrows = 2
ncols = 2
show_grid(df, nrows, ncols)

In [ ]:
!pip install monai

In [ ]:
from monai.transforms import (
    AddChanneld,
    LoadImage,
    LoadImaged,
    Orientationd,
    Rand3DElasticd,
    RandAffined,
    Spacingd,
)
from monai.config import print_config
from monai.apps import download_and_extract
import numpy as np
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

In [ ]:
from pathlib import Path
from typing import Callable
from typing import List
from typing import Optional
from typing import Tuple

import albumentations as A
#import cupy as cp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import seaborn as sns
#import segmentation_models_pytorch as smp
import torch
from albumentations.pytorch import ToTensorV2
from monai.metrics.utils import get_mask_edges
from monai.metrics.utils import get_surface_distance
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import Metric
from torchmetrics import MetricCollection
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [ ]:
from monai.transforms import RandGaussianNoise

transform = RandGaussianNoise(std = 200,mean =0 ,prob=1)

try_tranform(df,nrows,transform,image_flag = 1,mask_flag = 0)

In [ ]:
from monai.transforms import ShiftIntensity

transform = ShiftIntensity(offset = 0.5)

try_tranform(df,nrows,transform,image_flag = 1,mask_flag = 0)

In [ ]:
from monai.transforms import NormalizeIntensity

transform = NormalizeIntensity()

try_tranform(df,nrows,transform,image_flag = 1,mask_flag = 0)

In [ ]:
from monai.transforms import RandHistogramShift

transform = RandHistogramShift(num_control_points=10, prob=0.05)

try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 1)


In [ ]:
from monai.transforms import RandStdShiftIntensity

transform = RandStdShiftIntensity(100)

try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0)


In [ ]:
from monai.transforms import CropForeground

def threshold_at_one(x):
    # threshold at 1
    return x > 200
transform = CropForeground(select_fn=threshold_at_one, margin=0)

try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0)

In [ ]:
from monai.transforms import RandSpatialCrop

transform = RandSpatialCrop(roi_size=1)

try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0)


In [ ]:
from monai.transforms import Rand2DElastic
import albumentations as A

transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])


try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0,albumentation=1)

In [ ]:
# from monai.transforms import Rand2DElastic
# import albumentations as A

# transform = A.Compose([
#     A.augmentations.transforms.CLAHE(clip_limit=4.0,tile_grid_size=(8, 8),p=1)
# ])


# try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0,albumentation=1)

In [ ]:
from monai.transforms import HistogramNormalize

transform = HistogramNormalize()

try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0)



In [ ]:

from monai.transforms import Rand2DElastic
import albumentations as A

transform = A.Compose([
    A.augmentations.transforms.RandomBrightnessContrast(brightness_limit=[0,0,0], contrast_limit=[0,0,0], p=1)
])


try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0,albumentation=1)

In [ ]:
from monai.transforms import Rand2DElastic
import albumentations as A

transform = A.Compose([
    A.augmentations.transforms.GaussNoise(var_limit=(0.001, 0.02), mean=0, per_channel=False, p=1.0)
])


try_tranform(df,nrows,transform,image_flag =1 ,mask_flag = 0,albumentation=1)
